<a href="https://colab.research.google.com/github/SanchitaMahajan/Programming-Assignment/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ASSIGNMENT**

##By Sanchita Mahajan (20467876)

1. This notebook will prompt the user to input a company's stock market ticker symbol. 
2. It will then get the data from the Yahoo Finance website to extract information on the key of the company. 
3. Then the income tax will be calculated for each
executive based on the ATO tax tables. 
4. The User can further choose if the data
will be saved as a CSV and/or stored in an SQLite database.

##***Here is the link of the Yahoo Finance WebPage:-***

https://au.finance.yahoo.com/







##***Here is the ATO tables based on which Income tax of all Key Executives will be calculated:-***


[Resident tax rates 2021–22
Taxable income](https://www.ato.gov.au/rates/individual-income-tax-rates/)

Income            | Tax on this income 
------------------|----------------------
0 – \$18,200       | Nil
\$18,201 – \$45,000 | 19 cents for each \$1 over \$18,200
\$45,001 – \$120,000 | \$5,092 plus 32.5 cents for each \$1 over \$45,000
\$120,001 – \$180,000 |  \$29,467 plus 37 cents for each \$1 over \$120,000
\$180,001 and over | \$51,667 plus 45 cents for each \$1 over \$180,000

 # **Algorithm**

*1. Firstly, The user has to input a valid ticker symbol **(Only Australian Companies are allowed to search, use '.AX')**.*

*2. Ticker symbol will be used to extract Web page and scrape data of Key Executives.*

*3. A function will be created to get the URL.*

*4. Then a function will be created to extract cells data of each row.*

*5. Finally, The function will be created to get salary of all key executives and calculate tax on the pay based on the ATO tables.* 


# **User Instructions:-**
***1. Please input a Valid stock ticker symbol.***

***2.Only Australian companies are allowed to be searched.***

***3.Kindly use '.AX' after the company names to search for stock ticker symbols.*** 

***4.Dont use Numbers and use only valid stock ticker symbols like NAB.AX, COL.AX, ANZ.AX , CBA.AX and so on.***


# **First Step is to Install Unique Packages:-**
***1. We have to install the following packages..***

In [ ]:
# Install unique packages
!pip install kora -q

# load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd
import csv
import ipywidgets as widgets


# **Second Step is to Create a function to get the URL:-**
***1. The User has to input a stock ticker symbol..***

***2. The function will then return the Webpage of the input Stock ticker provided by the user and provide details of the Key Executive of the Stock ticker company.***

***3.The Input here is the stock ticker symbol which will get the URL of the profile table of the Yahoo Finance webpage.***



---




# **WORKING:-**

After careful analysis we realised that:-

**URL of NAB.AX:** https://au.finance.yahoo.com/quote/NAB.AX/profile?p=NAB.AX

**URL of COL.AX:** https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX

***The only difference is the name placement in the URL, therefore we can set up the URL in the following way:-***

  **example_url**  = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'

  ***(We only have to input the stock ticker symbols in the Curly brackets{}..)***

---



##*So,the result URL will be:-*

***example_url.format(ticker_symbol, ticker_symbol)*** 



In [ ]:
def get_url():
  '''This function will prompt the user to input a ticker symbol and get 
  the URL of the profile page'''
  
  ticker_symbol = input( "Please input a ticker symbol ")
  print(ticker_symbol)
  example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format(ticker_symbol, ticker_symbol)
  return result


#**Third Step is to Create a function to get the Cells data of the generated table:-**

***1. This function will return the Name, Title and Pay of all the Key Executives of the input stock ticker symbol company...***




---

# **Working:-**

***1. Cells can be find using row.find_all('td') where ('td') is used to get the cells details.*** 

***2. For the Key Executives,We just need the Name, title and pay of the excecutives. So, accordingly we just need text of 1,2 and 3 cells.*** 

In [ ]:
def get_cells_data(row):
  ''' This function will get the data of the first cells(Name, Title, Pay 
  of the Key Executives''' 
  
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title,'pay': pay}
  

# **Fourth Step is to Create a function to extract records of Key Executives of different ticker symbols:-**

---



# **Working:-**

***1. Table will be found using the soup.find('table).***

**2. Further table body will be found using table.find('tbody').**

**3. Atlast rows data will be extracted using body.find_all('tr').**   

**4. Result can be extracted using get_cells_data(row) where get_cells_data function was used to extract details of the first 3 cells of the rows (Name, Title and Pay).**

**5.Atlast this function will return the records of all the rows of the stock ticker symbol.**


In [ ]:
def get_extract_records(soup):
  '''This function will extract records of key executives of different 
 ticker symbols'''
 
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  extract_records = []
  for row in rows:
    result = get_cells_data(row)
    extract_records.append(result)
  return extract_records
  

# **Fifth Step is to Create a function to get the salary of the Key Executives:-**

This function will help to extract the salary of all the Key Executives from the table.

---



# **Working:-**

**1.After careful analysis,We saw that the pay of each executive is shown as 2.38M,9.988K and N/A and so on...**


**2.But to calculate the salary, we need to remove the (M, k) and convert the salary in AUD by multiplying the given figures with:-**

`(Pay*1000000), In case of M (millions)` 

`(Pay*1000), In case of K (thousands)`

**3.So, for this, the last pay character needs to be removed, by using the following:-**

`last_pay_character = pay[-1]`

**4.And to calculate the num_part if function is used saying if the last character of pay is 'M' then,** 

`salary = float(num_part) * 1000000`
    
  `elif last_pay_character == 'k':`
  
    salary = float(num_part) * 1000
  
  `else:`

   `salary = 0 (N/A)`





***This function will return the salary of all the executives in AUD....***
  


In [ ]:
def get_salary(rec):
  ''' This function will get the salary(pay) of the Key executives'''
  
  pay = rec['pay']
  last_pay_character = pay[-1]
  num_part = pay[0:-1]

  if last_pay_character == 'M':
    salary = float(num_part) * 1000000
    #print("Millions")
  elif last_pay_character == 'k':
    salary = float(num_part) * 1000
    #print("Thousand")
  else:
    salary = 0
    #print("Not sure")

  return salary
  

# **Sixth Step is to Create a function to Calculate the Income tax(based on ATO table) on salary of the Key Executives:-**

***1.This function will help in calculating the tax on the salaries of the key executives.***

***2.Income will be calculated based on ATO Tables:-***

`If income <= 18200, there will be zero tax`

`elif 18200<income<45000, tax will be calculated based on 0.19 8* (income - 18200)`

` elif 45000 < income < 120000, tax will be calculated based on 5092 + 0.32 * (income - 45000)`

`elif 120001 < income < 180000, tax will be calculated based on 29467 + 0.37 * (income - 120000)` 
   
`elif income > 180000,tax will be calculated based on 51667 + 0.45 * (income - 180000)`

***This function will return the tax payable by all key executives.***




In [ ]:
def calculate_tax(income):
   ''' This function will calculate the tax (based on ATO table) 
   on salary(pay) of the Key executives'''
   
   if income <= 18200:
    tax_payable = 0
   elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
   elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
   elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
   elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)
    
    return tax_payable
   

# **Final step is to scrape the Yahoo Finance Table of Key Executives which includes the following columns:-**

***1. Name of Key Executives.***

***2. Title of Key Executives.***

***3. Pay of Key Executives.***

***4. Tax of Key Executives.***

---






###*NOTE: PANDAS Dataframe has been used to display table in an attractive way! 

##`Only Australian Companies stock ticker symbols are allowed. So, Please use '.AX' after the stock ticker symbol.. `

In [ ]:
# Get URL

''' This will help to scrape the table of the stock ticker symbol input 
by the user and include tax column as well'''

url = get_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_extract_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# Create a pandas dataframe 

execs_df = pd.DataFrame(records)
execs_df


# **Saving the Output Results:-**

***1. CSV File***

***2. Excel File***

***3. Sqlite Database***


---


***The User can choose to save the data either in csv, excel or SQLite database or three of them together..***



**The User can save the output in CSV using the following cell:-**

***We just need to import csv module for this which is already done in Step 1.***

# ***1. Just run the cell using (Shift + Enter)***

In [ ]:
'''Saving the Income Tax of Key Executives in a csv and an excel file'''
# Saving the output
execs_df.to_csv('Key_Executives.csv')
execs_df.to_excel('Key_Executives.xlsx')
print('Saved the csv file, Please download it from files')

# ***Giving an Option to the User to save the output in csv or Sqlite database form:-***


*`The if and elif function have been used to provide an option to the User to choose if they want to store the output in csv file or Sqlite database file.`*

## ***Pandas provide an option to save the files in csv form and sqlite form.***

**Three Options have been created A, B and C.**

***1.Option A = 'CSV File'***

***2.Option B = 'SQLite Database***'





***3.Option C = input("Please select A OR B options from above to store the output :")***


---




**`if function is used saying if the User chooses Option C == 'A', Pandas dataframe will be converted to csv file and a message will pop up saying "Please download csv file from the files folder"`**.

 **`Further, elif the User chooses option C == 'B', sqlite3 will be import first. And a connection will be made between the sqlite3 (imported) with the database you want to create to store the income tax of Key Executives. A message will pop up saying that "Please download database from the files"!`**



---



---

### **1. I am getting an error message for Sqlite if after saving database for one stock ticker symbol and I re run all the cells, I get an Value Error message saying that execs_df table already exists!**

### **2. So, to resolve this issue I have to go to Edit Option and clear output for all the cells and And then go to Runtime and choose Factory Reset and run all cells again,then the database can be re saved!** 

In [ ]:
execs_df = pd.DataFrame(records)
execs_df.to_csv('Key_Executives.csv')
print('Key_Executives.csv')
A= 'CSV File' 
B= 'SQLite Database'
C= input("Please select A OR B options from above to store the output :")
if C == 'A':
  execs_df = pd.DataFrame(records)
  execs_df.to_csv('Key_Executives.csv')
  print('Key_Executives')
  print("Please download CSV from the files")
elif C == 'B':
  import sqlite3
  conn = sqlite3.connect("Key_Executives.db")
  execs_df.to_sql('execs_df',conn)

  print("Please download database from the files")

In [ ]:
Name= pd.read_sql('SELECT * FROM execs_df', conn)
Name.head()

# ***ALTERNATIVE CODE...***

##**I tried this code (Reference from Stack Overflow) but it went quiet long! So, I skipped it due to lack of time...**

***`Import OS (Os is operating system interface module of python to change file name or delete file once its downloaded in colab files)`***

I was also getting confused with few steps, so I left it due to lack of time....



In [ ]:


# Get URL

''' This will help to scrape the table of the stock ticker symbol input 
by the user and include tax column as well'''

url = get_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_extract_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# Create a pandas dataframe 

execs_df = pd.DataFrame(records)
execs_df

'''This is to save the output in csv or Sqlite database'''

execs_df = pd.DataFrame(records)
execs_df.to_csv('Key_Executives.csv')
print('Key_Executives.csv')
A= 'CSV File' 
B= 'SQLite Database'
C= input("Please select A OR B options from above to store the output :")
if C == 'A':
  execs_df = pd.DataFrame(records)
  execs_df.to_csv('Key_Executives.csv')
  print('Key_Executives')
  print("Please download CSV from the files")
  file = 'Key_Executives.csv'
  if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)
  else:
    print("file not found")
elif C == 'B':
  import sqlite3
  conn = sqlite3.connect("Key_Executives.db")
  cursor = conn.cursor()
  query = """
  CREATE TABLE Executives (
            name TEXT,
            title TEXT,
            Pay INTERGER,
            Tax INTERGER
        );"""
  execs_df = pd.read_csv('Key_Executives.csv')
  execs_df.to_sql('Executives', conn, if_exists='append', index=False)
  print("Please download database from the files")
  import os
  file = 'Key_Executives.csv'
  if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)
  else:
    print("file not found")
else:
  print("Select Correct Option")
  print("Displaying Database without Saving it into any file format")
print('Key_Executives')
cursor.execute("SELECT * from Executives")
X=cursor.fetchall()
dff=pd.DataFrame(X)
dff.columns =['SERIAL No.', 'NAME','Title','Pay','Tax']
dff.head()
